# Prospecção de Dados (Data Mining) DI/FCUL - HA2

## Second Home Assignement (MC/DI/FCUL - 2024)

### Fill in the section below

### GROUP:`01` 

* Catherine Prokhorov (62608) - () Hours worked on the project
* Guilherme Cepeda (62931) - () Hours worked on the project* Jorge Aleluia (54549) - () Hours worked on the project * 
Rómulo Nogueira (56935) -()  Hours worked on the project



## Objectives
The purpose of this Home Assignment is:

1. Analyze the itemset/rules generation procedure
2. Process and identify the most relevant rules

**NOTE 1: Students are not allowed to add more cells to the notebook**

**NOTE 2: The notebook must be submited fully executed**


### 1. Read the Dataset

Dataset: Instacart [Market Basket Analysis](https://www.kaggle.com/datasets/psparks/instacart-market-basket-analysis)



The purpose of this Home Assignment is:

1. Analyze the itemset/rules generation procedure
2. Identify the most relevant rules

Please download the HA files from this moodle folder and inspect it like this:
**NOTE 1: Students are not allowed to add more cells to the notebook**

**NOTE 2: The notebook must be submited fully executed**

The dataset has been preprocessed with transactions as lists of integers. It is necessary a decoder to see the actual products bought together

Uncompress the data set (zip file) which will produce 2 files:
* `order_products.pickle` a Python pickle with all the trasactions
* A `products.txt` - decoder table with all the data relative to each item



In [1]:
#Read product names and IDs
lines=open("products.txt", "rt", encoding="utf8").readlines()
products=[0]*len(lines)
for lin in lines[1:]:
    pid, pname, aid, did=lin.strip().split("\t")    
    products[int(pid)]=pname
    
#read transactions
import pickle
orders=pickle.load(open("order_products.pickle", "rb"))

#check names of products on transaction 2 (example):
print("Transaction 2 is", orders[2])
for prod in orders[2]: print("Code: %5d is product: %s" %(prod, products[prod]))

# # limit for the number of products to display
# product_limit = 10000
# product_count = 0

# # Check names of products on all transactions
# for transaction_id, transaction_products in orders.items():
#     print(f"Transaction {transaction_id}: {transaction_products}")
#     for prod in transaction_products:
#         product_name = products[prod]
#         if product_name:
#             print(f"Code: {prod} is product: {product_name}")
#             product_count += 1
#             if product_count >= product_limit:
#                 print("Limite Alcançado")
#                 break
#         else:
#             print(f"No product found for code: {prod}")
#     if product_count >= product_limit:
#         break


Transaction 2 is [33120, 28985, 9327, 45918, 30035, 17794, 40141, 1819, 43668]
Code: 33120 is product: Organic Egg Whites
Code: 28985 is product: Michigan Organic Kale
Code:  9327 is product: Garlic Powder
Code: 45918 is product: Coconut Butter
Code: 30035 is product: Natural Sweetener
Code: 17794 is product: Carrots
Code: 40141 is product: Original Unflavored Gelatine Mix
Code:  1819 is product: All Natural No Stir Creamy Almond Butter
Code: 43668 is product: Classic Blend Cole Slaw


### Objective 1 - Analyze the itemset/rules generation procedure

1. From the approaches used in classes make a performance analysis up to a threshold level of support
2. Define a good support threshold for analysis according to your computational capabilities


In [2]:
### Add supporting functions here
from itertools import combinations

def getSupport(items, Trs):
    N=len(Trs)
    R=[i for i in range(N) if items.issubset(Trs[i])]
    return len(R)/N

def getAllItems(Trs):
    res=set()
    for t in Trs: res=res | t
    return res

def check_support(support_sets, min_support, database):
    D={}
    for ss in support_sets:
        supp = getSupport(ss, database)
        if supp >= min_support: D[ss]=supp
    return D

def combine(support_sets):
    D={}
    SS = list(support_sets.keys())
    for i in range(len(SS)-1):
        for j in range(i+1, len(SS)):
            new_ss = SS[i] | SS[j]
            if len(new_ss)==len(SS[i])+1:
                D[frozenset(new_ss)]=0 # do not initialize support for now
    return D    

def prune(L1, L0):
    #we must check whether all combinations of L1 are in L
    D={}
    for support_set in L1:
        sub_sets = [frozenset(c) for c in combinations(support_set, len(support_set) - 1)] 
        #print("####  ", support_set, sub_sets)
        add_ss=True
        for sub_set in sub_sets:
            if sub_set not in L0: # L0 previous layer
                add_ss=False
                break
        if add_ss==True: D[support_set] = L1[support_set]
    return D

def PD_apriori(tr_sets, min_support):
    min_support_sets={}
    items=getAllItems(tr_sets)
    L0 = {frozenset([item]): getSupport({item}, tr_sets) for item in items}
    #prune Level 0 according to min support
    L0 = {s: L0[s]for s in L0 if L0[s]>=min_support}
    min_support_sets.update(L0)
    while len(L0)>1:
        L1 = combine(L0)
        L1 = prune(L1, L0)
        L1 = check_support(L1, min_support, tr_sets)
        if len(L1) >0: min_support_sets.update(L1)
        L0 = L1
    return min_support_sets

In [24]:
### Add processing code here
transaction_2_ids = orders[2]
transaction_2_products = [products[prod_id] for prod_id in transaction_2_ids]

min_supp=1
min_supp_sets= PD_apriori([set(transaction_2_products)], min_supp)
for ss in min_supp_sets: print(list(ss), " has support =", min_supp_sets[ss])

['Michigan Organic Kale']  has support = 1.0
['Coconut Butter']  has support = 1.0
['Carrots']  has support = 1.0
['Organic Egg Whites']  has support = 1.0
['Garlic Powder']  has support = 1.0
['Classic Blend Cole Slaw']  has support = 1.0
['All Natural No Stir Creamy Almond Butter']  has support = 1.0
['Original Unflavored Gelatine Mix']  has support = 1.0
['Natural Sweetener']  has support = 1.0
['Coconut Butter', 'Michigan Organic Kale']  has support = 1.0
['Carrots', 'Michigan Organic Kale']  has support = 1.0
['Organic Egg Whites', 'Michigan Organic Kale']  has support = 1.0
['Garlic Powder', 'Michigan Organic Kale']  has support = 1.0
['Classic Blend Cole Slaw', 'Michigan Organic Kale']  has support = 1.0
['All Natural No Stir Creamy Almond Butter', 'Michigan Organic Kale']  has support = 1.0
['Original Unflavored Gelatine Mix', 'Michigan Organic Kale']  has support = 1.0
['Natural Sweetener', 'Michigan Organic Kale']  has support = 1.0
['Coconut Butter', 'Carrots']  has support 

### Your short analysis here

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum

### Objective 2 - Identify the most relevant rules

1. From your predefined support level generate all available itemsets and generate rules
2. Identify a set of 10 relevant rules using the Highest Lift criterion
    * **NOTE**: Present the rules with the product names and not as Integers 
3. Identify the Maximal and Closed Itemsets for the same level of support and generate 5 rules using the Highest Lift
    * **NOTE**: Do not list the Maximal or Closed Itemsets. Present just a few as an example and mention **how many** Closed and Maximal Itemsets were found for the selected support level
    * **NOTE**: Present the rules with the product names and not as Integers 


In [30]:
### Add supporting functions here
def getConfidence(X, Y, Trs):
    # confidence(X=>Y) = support(X U Y)/support(X)
    supp_X=getSupport(X, Trs)
    if supp_X == 0:
        return 0 
    supp_XUY = getSupport(X | Y, Trs)
    return supp_XUY / supp_X if supp_X != 0 else 0
    
def checkRules(itemset, Trs, conf_thres):
    rules=[]
    for i in itemset:
        Y={i}
        X=itemset - Y
        if len(X)>0:
            conf=getConfidence(X, Y, Trs)
            if conf>=conf_thres:
                rules.append((X, Y, conf))
    return rules

def getAllRules(MS_itemsets, Trs, conf_level):
    allrules=[]
    for itemset in MS_itemsets:
        R=checkRules(set(itemset), Trs, conf_level)
        allrules+=R
    return allrules


In [32]:
### Add processing code here
transaction_2_ids = orders[2]
transaction_2_products = [products[prod_id] for prod_id in transaction_2_ids]

for i, transaction in enumerate(transaction_2_products[:10]):
    print(f"Transaction {i + 1}: {transaction}")
    
rules=checkRules(set(transaction_2_products), transaction_2_products, 0.6)

## TODO()
for X,Y,conf in rules:
    print(X, "=>", Y, "\t[confidence: %6.4f]" % conf)

rules= getAllRules(min_supp_sets.keys(), transaction_2_products, 0.6)
for X,Y,conf in rules:
    print(X, "=>", Y, "\n[confidence: %6.4f] [support: %6.4f]\n" % (conf, min_supp_sets[frozenset(X | Y)]))


Transaction 1: Organic Egg Whites
Transaction 2: Michigan Organic Kale
Transaction 3: Garlic Powder
Transaction 4: Coconut Butter
Transaction 5: Natural Sweetener
Transaction 6: Carrots
Transaction 7: Original Unflavored Gelatine Mix
Transaction 8: All Natural No Stir Creamy Almond Butter
Transaction 9: Classic Blend Cole Slaw


### Your short analysis here

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum
